In [17]:
import IPython.display
import numpy as np

<img src="bracket.png">

There’s a certain insanity in the air this time of the year that gets us thinking about tournament brackets. Consider a tournament with 16 competitors, seeded 1-16, and arranged in the single-elimination bracket pictured above (identical to a “region” of the NCAA Division 1 basketball tournament). Assume that when the X-seed plays the Y-seed, the X-seed has a Y/(X+Y) probability of winning. E.g. in the first round, the 5-seed has a 12/17 chance of beating the 12-seed.

Suppose the 2-seed has the chance to secretly swap two teams’ placements in the bracket before the tournament begins. So, for example, say they choose to swap the 8- and 16-seeds. Then the 8-seed would play their first game against the 1-seed and have a 1/9 chance of advancing to the next round, and the 16-seed would play their first game against the 9-seed and have a 9/25 chance of advancing.

What seeds should the 2-seed swap to maximize their (the 2-seed’s) probability of winning the tournament, and how much does the swap increase that probability? Give your answer to six significant figures.


In [18]:
class Node:
    def __init__(self, key):
        self.key = key
        self.right = None
        self.left = None

class JS_Bracketology:
    def __init__(self, player):
        assert (player <= 16) and (player >0) and isinstance(player, int)
        self.n = 16
        self.m = int(self.n / 2)
        self.player = player
        self.l = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]
        self.ll = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]
        self.playeridx = self.l.index(player)

        self.out = {}

    def proba(self, a, b):
        return b / (a + b)

    def f(self, x, y, i):
        l = y[i].key['L']
        r = y[i].key['R']
        s = [a for a in [*x[2 * i + 1].key] if isinstance(a, int)]
        q = [a for a in [*x[2 * i].key] if isinstance(a, int)]
        
        for a in l:
            y[i].key[a] = x[2 * i].key[a] * sum([x[2 * i + 1].key[k] * self.proba(a, k) for k in s])
        for a in r:
            y[i].key[a] = x[2 * i + 1].key[a] * sum([x[2 * i].key[k] * self.proba(a, k) for k in q])

    def succes_probas(self):
        self.x = [Node(
                {self.l[2 * i]: self.proba(self.l[2 * i], self.l[2 * i + 1]),
                self.l[2 * i + 1]: self.proba(self.l[2 * i + 1], self.l[2 * i])}
                ) for i in  range(int(len(self.l) / 2))]
        
        self.y = [Node({'L': [*self.x[2 * i].key],
                        'R': [*self.x[2 * i + 1].key]}) 
                  for i in range(int(len(self.l) / 4))]

        for i in range(len(self.y)):
            self.f(self.x, self.y, i)
            
        z = [Node({'L': self.y[2 * i].key['L'] + self.y[2 * i].key['R'],
                   'R': self.y[2 * i + 1].key['L'] + self.y[2 * i + 1].key['R']}) 
             for i in range(int(len(self.l) / 8))]
        
        for i in range(len(z)):
            self.f(self.y, z, i)
            
        aa = [Node({'L': z[2 * i].key['L'] + z[2 * i].key['R'], 
                    'R': z[2 * i + 1].key['L'] + z[2 * i + 1].key['R']})
              for i in range(int(len(self.l) / 16))]
        
        for i in range(len(aa)):
            self.f(z, aa, i)

        self.out[tuple(self.l)] = aa[0].key[self.player]
        del z, aa, self.x, self.y

    def relace_and_run(self, index_from, index_to):
        self.l = self.ll.copy()
        tmp = self.l[index_from]
        self.l[index_from] = self.l[index_to]
        self.l[index_to] = tmp
        self.succes_probas()

    def run(self):
        self.succes_probas()
        for i in  (range(self.m )):
            for j in range(i + 1, self.m):
                self.relace_and_run(2 * i, 2 * j)
                self.relace_and_run(2 * i, 2 * j + 1)
                self.relace_and_run(2 * i + 1, 2 * j)
                self.relace_and_run(2 * i + 1, 2 * j + 1)

        d = sorted(self.out.items(), key=lambda x: x[1])
        print('Optimal swap proba of player:', self.player, ' wins is', d[-1][1])
        print('Initial game proba of player:', self.player, ' wins is', self.out[tuple(game.ll)])
        print('Proba gain:', d[-1][1] - self.out[tuple(game.ll)])
        tmp=np.array(d[-1][0])-np.array(self.ll)
        ind=np.where(tmp!=0)
        print('Tournament initial allure',self.ll )
        print('Tournament allure',d[-1][0] )
        print('The swap :', np.array(self.ll)[ind] )

In [19]:
%%time
game=JS_Bracketology(2)
game.run()

Optimal swap proba of player: 2  wins is 0.2816191915195929
Initial game proba of player: 2  wins is 0.2160396878170165
Proba gain: 0.06557950370257642
Tournament initial allure [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]
Tournament allure (1, 3, 8, 9, 5, 12, 4, 13, 6, 11, 16, 14, 7, 10, 2, 15)
The swap : [16  3]
Wall time: 19.7 ms
